# Homework 3

**Name:** -- Roberto José González --

**e-mail:** -- roberto.jose0745@alumnos.udg.mx --

# MODULES

In [14]:
# Load modules
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from scipy.stats import cauchy, pareto

# Productivity seed configuration
np.random.seed(42)

print("Enviroment configurated and modules loaded")


Enviroment configurated and modules loaded


# **Activity 1:** Path length - (BM1 vs BM2 vs CRW)
This activity generates two types of trajectories: Brownian Motion (BM), created by summing normal random steps, and Correlated Random Walk (CRW), where each step depends on the previous angle plus a random variation. The total length of each trajectory is calculated by summing the distances between consecutive points. The code implements functions to generate these trajectories and calculate their length, visualizing them in an interactive graph.

In [15]:
# Brownian Motion trejectory
def generate_bm_trajectory(n_steps=100, step_std=1.0):
    # (x,y) randon walk generation
    steps_x = np.random.normal(loc=0, scale=step_std, size=n_steps)
    steps_y = np.random.normal(loc=0, scale=step_std, size=n_steps)
    # Acumulated random walk 
    x = np.cumsum(steps_x)
    y = np.cumsum(steps_y)
    return pd.DataFrame({'x': x, 'y': y})

# Correlated Random Walk trajectory 
def generate_crw_trajectory(n_steps=100, step_length=1.0, turning_angle_std=np.pi/8):
    # Initial random angle
    angle = np.random.uniform(0, 2*np.pi)
    x, y = [0], [0]
    for _ in range(n_steps):
        # Normal distribution added
        delta_angle = np.random.normal(loc=0, scale=turning_angle_std)
        angle += delta_angle
        # Position updated
        x.append(x[-1] + step_length * np.cos(angle))
        y.append(y[-1] + step_length * np.sin(angle))
    # It¿nitial point skipped 
    return pd.DataFrame({'x': x[1:], 'y': y[1:]})

# Trajectory length calculation
def compute_path_length(traj_df):
    # Euclidian distance calculation
    dx = np.diff(traj_df['x'])
    dy = np.diff(traj_df['y'])
    distances = np.sqrt(dx**2 + dy**2)
    return np.sum(distances)

# Trajectories generated
bm1 = generate_bm_trajectory(n_steps=20, step_std=1.0)
bm2 = generate_bm_trajectory(n_steps=20, step_std=1.0)
crw = generate_crw_trajectory(n_steps=20, step_length=1.0, turning_angle_std=np.pi/8)

# Length calculation of each trajectory
length_bm1 = compute_path_length(bm1)
length_bm2 = compute_path_length(bm2)
length_crw = compute_path_length(crw)

print("BM1 length:", length_bm1)
print("BM2 length:", length_bm2)
print("CRW length:", length_crw)

# Trajectory graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=bm1['x'], y=bm1['y'], mode='lines', name=f'BM1 (Long: {length_bm1:.2f})'))
fig.add_trace(go.Scatter(x=bm2['x'], y=bm2['y'], mode='lines', name=f'BM2 (Long: {length_bm2:.2f})'))
fig.add_trace(go.Scatter(x=crw['x'], y=crw['y'], mode='lines', name=f'CRW (Long: {length_crw:.2f})'))
fig.update_layout(title="Trajectories: BM1 vs BM2 vs CRW",
                  xaxis_title="X", yaxis_title="Y")
fig.show()


BM1 length: 22.446900109144245
BM2 length: 22.43894568652474
CRW length: 19.0


# **Activity 2:** Mean Squared Displacement - (BM vs CRW)
The Mean Squared Displacement (MSD) is a statistical measure that quantifies particle dispersion over time. It's calculated by averaging the squared distance traveled for different lag times. The MSD varies between Brownian Motion (BM) trajectories, which exhibit linear growth, and Correlated Random Walk (CRW) trajectories, where correlation between steps influences the MSD. The code implements a function to calculate the MSD and visually compares BM and CRW trajectories in an interactive graph.

In [16]:
#MSD function 
def compute_MSD(traj_df):
    n = len(traj_df)
    msd = []
    # Calulation for each tau
    for tau in range(1, n):
        displacements = (traj_df['x'][tau:].values - traj_df['x'][:-tau].values)**2 + \
                        (traj_df['y'][tau:].values - traj_df['y'][:-tau].values)**2
        msd.append(np.mean(displacements))
    return np.array(msd)

# Comparison BM and CRW trajectory
bm = generate_bm_trajectory(n_steps=300, step_std=1.0)
crw_trajectory = generate_crw_trajectory(n_steps=300, step_length=1.0, turning_angle_std=np.pi/8)

# MSD calculation for each one
msd_bm = compute_MSD(bm)
msd_crw = compute_MSD(crw_trajectory)

# Data preparation
tau_values = np.arange(1, len(msd_bm)+1)

#Graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=tau_values, y=msd_bm, mode='lines', name='MSD BM'))
fig.add_trace(go.Scatter(x=tau_values, y=msd_crw, mode='lines', name='MSD CRW'))
fig.update_layout(title="Mean Squared Displacement (MSD) BM vs CRW",
                  xaxis_title="Tau (offset)",
                  yaxis_title="MSD")
fig.show()


# **Activity 3:** Turning-angle Distribution - (source dist. vs observed dist.)
This fragment describes the analysis of turning angles in a Correlated Random Walk (CRW). Turning angles, which represent changes in direction between consecutive steps, are calculated and compared to a theoretical Cauchy distribution. The code generates CRW trajectories where turning angles follow a Cauchy distribution, calculates the actual turning angles, and visualizes them in a histogram, overlaying the theoretical probability density for comparison.

In [17]:
# CRW generation function with Cauchy distribution
def generate_crw_trajectory_cauchy(n_steps=100, step_length=1.0, cauchy_scale=0.2):
    angle = np.random.uniform(0, 2*np.pi)
    x, y = [0], [0]
    for _ in range(n_steps):
        # Turning angle of Cauchy distribution
        delta_angle = cauchy.rvs(loc=0, scale=cauchy_scale)
        angle += delta_angle
        x.append(x[-1] + step_length * np.cos(angle))
        y.append(y[-1] + step_length * np.sin(angle))
    return pd.DataFrame({'x': x[1:], 'y': y[1:]})

# Trajectory turning angle extraction function
def get_turning_angles(traj_df):
    # Angle of each step calculation
    angles = np.arctan2(np.diff(traj_df['y']), np.diff(traj_df['x']))
    # Angle difference calculation
    turning_angles = np.diff(angles)
    # Angle rage adjustment (-pi, pi)
    turning_angles = (turning_angles + np.pi) % (2*np.pi) - np.pi
    return turning_angles

# Two CRW trajectories with diferent Cauchy scalas
crw_cauchy_1 = generate_crw_trajectory_cauchy(n_steps=100000, step_length=1.0, cauchy_scale=0.6)
crw_cauchy_2 = generate_crw_trajectory_cauchy(n_steps=100000, step_length=1.0, cauchy_scale=0.9)

# Turning angles
angles_1 = get_turning_angles(crw_cauchy_1)
angles_2 = get_turning_angles(crw_cauchy_2)

# Histograms and theorical density function (Cauchy)
x_range = np.linspace(-np.pi, np.pi, 4000)
pdf_1 = cauchy.pdf(x_range, loc=0, scale=0.6)
pdf_2 = cauchy.pdf(x_range, loc=0, scale=0.9)

fig = go.Figure()
# First trajectory histogram
fig.add_trace(go.Histogram(x=angles_1, histnorm='probability density',
                           name='Observed (Scale=0.6)', opacity=0.6))
# First trajectory theorical function
fig.add_trace(go.Scatter(x=x_range, y=pdf_1, mode='lines',
                         name='Cauchy (Scale=0.6)'))

# Second trajectory histogram
fig.add_trace(go.Histogram(x=angles_2, histnorm='probability density',
                           name='Observed (Scale=0.9)', opacity=0.6))
# Second trajectory theorical function
fig.add_trace(go.Scatter(x=x_range, y=pdf_2, mode='lines',
                         name='Cauchy (Scale=0.9)'))

fig.update_layout(title="Turning angles distribution: Observed vs Theoretical",
                  xaxis_title="Turning angle (radians)",
                  yaxis_title="Probability density",
                  barmode='overlay')
fig.show()


# **Activity 4:** Step-length Distribution - (source dist. vs observed dist.)
This fragment describes the Lévy Walk (LW), a type of trajectory where step lengths follow a heavy-tailed distribution, typically a power law (Pareto distribution). The alpha parameter controls the probability of long steps. The code generates LW trajectories by drawing step lengths from a Pareto distribution and random directions. It then calculates the actual step lengths and compares them with the theoretical Pareto distribution using histograms and overlaying the probability density function

In [39]:
# LÉVY WALK (LW) TRAJECTORY GENERATION
def generate_levy_walk_trajectory(n_steps=200, alpha=1.5, step_scale=1.0, turning_angle_std=np.pi/4):
    """
    Generates a Lévy Walk trajectory using a Pareto distribution (xm=1) 
    for step lengths and random turns with a normal distribution.
    """
    # Generate step lengths using Pareto:
    # np.random.pareto(alpha) -> [0, ∞); +1 ensures x >= 1.
    step_lengths = (np.random.pareto(alpha, size=n_steps) + 1) * step_scale
    
    # Initial random angle
    angle = np.random.uniform(0, 2 * np.pi)
    
    # Starting coordinates
    x, y = [0], [0]
    
    # For each step, turn and move
    for step in step_lengths:
        # Random turn with normal variation
        delta_angle = np.random.normal(loc=0, scale=turning_angle_std)
        angle += delta_angle
        
        # Update position
        x.append(x[-1] + step * np.cos(angle))
        y.append(y[-1] + step * np.sin(angle))
        
    # Return a DataFrame with the coordinates
    return pd.DataFrame({'x': x[1:], 'y': y[1:]})

# GET STEP LENGTHS

def get_step_lengths(traj_df):
    """
    Calculates the distance between consecutive points (dx, dy) 
    to obtain the length of each step in the trajectory.
    """
    dx = np.diff(traj_df['x'])
    dy = np.diff(traj_df['y'])
    return np.sqrt(dx**2 + dy**2)

# GENERATE TWO LW TRAJECTORIES WITH DIFFERENT ALPHA VALUES
lw1 = generate_levy_walk_trajectory(n_steps=300, alpha=1.5, step_scale=1.0, turning_angle_std=np.pi/4)
lw2 = generate_levy_walk_trajectory(n_steps=300, alpha=2.5, step_scale=1.0, turning_angle_std=np.pi/4)

# OBTAIN STEP LENGTHS
steps_lw1 = get_step_lengths(lw1)
steps_lw2 = get_step_lengths(lw2)

# CREATE AN EXTENDED RANGE TO SHOW THE COMPLETE PDF
# Take the maximum observed value from both trajectories
max_val = max(steps_lw1.max(), steps_lw2.max())

# Define a range from 0 to a higher value (e.g., 50 or max_val)
x_vals = np.linspace(0, max(50, max_val), 500)

# CALCULATE THE PARETO PDF, FORCING 0 WHEN x < 1
pdf_lw1 = np.where(
    x_vals < 1,
    0,  # PDF = 0 for x < 1
    1.5 * 1**1.5 / (x_vals ** (1.5 + 1))  # alpha=1.5, xm=1
)

pdf_lw2 = np.where(
    x_vals < 1,
    0,  # PDF = 0 for x < 1
    2.5 * 1**2.5 / (x_vals ** (2.5 + 1))  # alpha=2.5, xm=1
)

fig = go.Figure()

# Histogram for LW1
fig.add_trace(
    go.Histogram(
        x=steps_lw1, 
        histnorm='probability density',
        name='LW observed lengths (alpha=1.5)', 
        opacity=0.6,
        xbins=dict(
            start=min(steps_lw1),
            end=max(steps_lw1),
            size=0.6  # value to change the bin width
        )
    )
)
# Theoretical PDF for LW1
fig.add_trace(
    go.Scatter(
        x=x_vals, 
        y=pdf_lw1, 
        mode='lines',
        name='Theoretical PDF (alpha=1.5)'
    )
)

# Histogram for LW2
fig.add_trace(
    go.Histogram(
        x=steps_lw2, 
        histnorm='probability density',
        name='LW observed lengths (alpha=2.5)', 
        opacity=0.6,
        xbins=dict(
            start=min(steps_lw2),
            end=max(steps_lw2),
            size=0.5 
        )
    )
)
# Theoretical PDF for LW2
fig.add_trace(
    go.Scatter(
        x=x_vals, 
        y=pdf_lw2, 
        mode='lines',
        name='Theoretical PDF (alpha=2.5)'
    )
)

fig.update_layout(
    title="Lévy Walk Step-Length Distribution: Observed vs Theoretical",
    xaxis_title="Step length",
    yaxis_title="Probability density",
    barmode='overlay'
)

fig.show()


C:\Users\rober\AppData\Local\Temp\ipykernel_836\3784301949.py:60: RuntimeWarning:

divide by zero encountered in divide

C:\Users\rober\AppData\Local\Temp\ipykernel_836\3784301949.py:66: RuntimeWarning:

divide by zero encountered in divide

